한글프롬프트만 반영, 에폭수 5까지

In [4]:
!pip install -q transformers evaluate sentencepiece
!pip install -q evaluate rouge_score
!pip install -q --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 67.6 MB/s eta 0:00:00


In [1]:
import torch
print("CUDA:", torch.cuda.is_available(), "| Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

CUDA: True | Device: Tesla T4


In [2]:
# ─── Colab 셀 3: 파일 업로드 (또는 Drive 마운트 후 경로 지정) ───
from google.colab import files
uploaded = files.upload()   # .jsonl 파일 하나만 선택하세요
file_path = next(f for f in uploaded if f.endswith(".jsonl"))
print("Raw data file:", file_path)

Saving 순화표현모델 데이터_리라이팅완료.jsonl to 순화표현모델 데이터_리라이팅완료.jsonl
Raw data file: 순화표현모델 데이터_리라이팅완료.jsonl


In [5]:
# ─── Colab 셀 4: 라이브러리 임포트 & 모델 로드 ───
import json
import numpy as np
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
import evaluate

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

MODEL_NAME = "wisenut-nlp-team/KoT5-base"
tokenizer  = AutoTokenizer.from_pretrained(MODEL_NAME)
model      = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [6]:
# ─── Colab 셀 5: 데이터 로드 & 한글 프롬프트 전처리 ───
with open(file_path, "r", encoding="utf-8") as f:
    raw_data = [json.loads(line) for line in f if line.strip()]
print("총 샘플 수:", len(raw_data))

def preprocess_korean(example):
    ctx   = example["context"][:-1]
    last  = example["context"][-1]
    prefix = "마지막 문장을 공손하게 바꿔줘:"   # 한글 지시문
    text = " ".join([prefix] + ctx + [last])
    return {"input_text": text, "target_text": example["output"]}

processed = [preprocess_korean(d) for d in raw_data]
print("샘플 전처리 예시:", processed[0])

총 샘플 수: 3100
샘플 전처리 예시: {'input_text': '마지막 문장을 공손하게 바꿔줘: A: 오늘 지각할 뻔했다ㅋㅋ 버스 진짜 안 옴 B: 나도 거의 뛰어서 옴;; A: 수학쌤 오늘 또 지옥 시간일 듯 B: 그 ㅆㅂ 늙은이 목소리만 들어도 현타 옴', 'target_text': 'B: 수학쌤 목소리 너무 지루하긴 해'}


In [7]:
# ─── Colab 셀 6: Dataset 정의 & train/test split ───
from sklearn.model_selection import train_test_split

train_list, test_list = train_test_split(processed, test_size=0.1, random_state=42)

class RewriteDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=256):
        self.data      = data
        self.tokenizer = tokenizer
        self.max_len   = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        item = self.data[i]
        enc = self.tokenizer(
            item["input_text"],
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        lbl = self.tokenizer(
            text_target=item["target_text"],
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )["input_ids"].squeeze()
        lbl[lbl == self.tokenizer.pad_token_id] = -100

        batch = {k: v.squeeze() for k, v in enc.items()}
        batch["labels"] = lbl
        return batch

train_ds = RewriteDataset(train_list, tokenizer)
test_ds  = RewriteDataset(test_list,  tokenizer)
collator = DataCollatorForSeq2Seq(tokenizer, model)

print("Train/Test 크기:", len(train_ds), len(test_ds))

Train/Test 크기: 2790 310


In [8]:
# ─── Colab 셀 7: 평가 함수 정의 ───
import numpy as np
import torch

bleu_metric  = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # tensor → numpy
    if isinstance(preds, torch.Tensor):  preds  = preds.detach().cpu().numpy()
    if isinstance(labels, torch.Tensor): labels = labels.detach().cpu().numpy()
    # mask 처리
    preds  = np.where(preds  == -100, tokenizer.pad_token_id, preds)
    labels = np.where(labels == -100, tokenizer.pad_token_id, labels)
    # decode
    dec_preds  = tokenizer.batch_decode(preds,  skip_special_tokens=True)
    dec_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # 지표
    bleu  = bleu_metric.compute(predictions=dec_preds, references=[[l] for l in dec_labels])["bleu"]
    rouge = rouge_metric.compute(predictions=dec_preds, references=dec_labels, use_stemmer=True)["rougeL"]
    return {"bleu": bleu, "rougeL": rouge}

print("compute_metrics 준비 완료")

compute_metrics 준비 완료


In [10]:
# ─── Colab 셀 8: Trainer 생성 (평가는 모두 제거) ───
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

args = Seq2SeqTrainingArguments(
    output_dir="/content/kot5-korprompt",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    generation_max_length=64,
    generation_num_beams=4,
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    warmup_steps=100,
    save_total_limit=2,
    lr_scheduler_type="linear",
    logging_dir="/content/logs_korprompt",
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    # eval_dataset와 compute_metrics를 빼고 학습만 수행
    tokenizer=tokenizer,
    data_collator=collator
)

print("Trainer 준비 완료 (평가 옵션 제거)")

Trainer 준비 완료 (평가 옵션 제거)


/tmp/ipython-input-1027049858.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [11]:
# ─── Colab 셀 9: 학습 시작 & 모델 저장 ───
trainer.train()
trainer.save_model("/content/kot5-korprompt")
tokenizer.save_pretrained("/content/kot5-korprompt")
print("학습 완료, 모델 저장됨")

/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss
500,5.274400
1000,4.389200
1500,3.753100
2000,3.293800
2500,2.998800
3000,2.814900


학습 완료, 모델 저장됨


In [12]:
# 수동 평가 예시
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate

# 1) 모델/토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("/content/kot5-korprompt")
model     = AutoModelForSeq2SeqLM.from_pretrained("/content/kot5-korprompt").to(device)

# 2) metric 로드
bleu_metric  = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

# 3) 예측 & 지표 계산
preds, refs = [], []
for ex in test_list:
    out = model.generate(
        **tokenizer(ex["input_text"], return_tensors="pt", truncation=True, max_length=256).to(device),
        max_length=64, num_beams=4
    )[0]
    preds.append(tokenizer.decode(out, skip_special_tokens=True))
    refs.append(ex["target_text"])

bleu = bleu_metric.compute(predictions=preds, references=[[r] for r in refs])["bleu"]
rouge = rouge_metric.compute(predictions=preds, references=refs, use_stemmer=True)["rougeL"]
print(f"수동 평가 → BLEU: {bleu:.4f}, ROUGE-L: {rouge:.4f}")

수동 평가 → BLEU: 0.0346, ROUGE-L: 0.9543


In [24]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 1) 디바이스 및 모델 로드 (로컬 경로에서만)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
FINETUNED_DIR = "/content/kot5-korprompt"   # 실제 모델 폴더 경로

tokenizer = AutoTokenizer.from_pretrained(
    FINETUNED_DIR,
    local_files_only=True   # ★ 로컬만 사용
)
model = AutoModelForSeq2SeqLM.from_pretrained(
    FINETUNED_DIR,
    local_files_only=True   # ★ 로컬만 사용
).to(device)

# 2) 입력 받기
print("대화 맥락을 줄 단위로 입력하세요. 빈 줄만 입력하면 종료됩니다.")
lines = []
while True:
    line = input()
    if not line.strip():
        break
    lines.append(line.strip())

if len(lines) < 2:
    print("최소 2줄 이상의 대화를 입력해야 합니다.")
    exit()

# 3) 포맷팅
prefix = "마지막 문장을 공손하게 바꿔줘:"
input_text = " ".join([prefix] + lines)

# 4) 토크나이즈 → 생성
inputs = tokenizer(
    input_text,
    return_tensors="pt",
    truncation=True,
    max_length=256
)
inputs = {k: v.to(device) for k,v in inputs.items()}

outputs = model.generate(
    **inputs,
    max_length=64,
    num_beams=4,
    early_stopping=True
)

# 5) 결과 출력
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n=== 입력 대화 ===")
for l in lines:
    print(l)
print("\n=== 순화 결과 ===")
print(result)

대화 맥락을 줄 단위로 입력하세요. 빈 줄만 입력하면 종료됩니다.
A: 아 너무 힘들어
A: 요즘 돈이 없어
B: 진짜?
B: 근데 너 경제관념이 원래 없는 거지잖아 ㅋㅋ


=== 입력 대화 ===
A: 아 너무 힘들어
A: 요즘 돈이 없어
B: 진짜?
B: 근데 너 경제관념이 원래 없는 거지잖아 ㅋㅋ

=== 순화 결과 ===
B: 경제관념이 부족한 것 같아
